# Measurement of Loudspeaker to Sequential Spherical Microphone Array

This notebook sketches how to sequentially measure impulse responses of spherical microphone arrays (SMAs) with the VariSphere device.

* Two sweeps are played, one for the loudspeaker and one for the analog feedback. 
* Two channels are recorded, the SMA microphone and the analog feedback.
* After each measurement, the VariSphere rotates to the next microphone position on the SMA grid.

## Load requirements

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2

import sys
from datetime import datetime, timedelta
from pathlib import Path
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
from IPython.display import Audio, display, Markdown
from ipywidgets import Output
from numpy.fft import rfft, rfftfreq
from scipy.io import savemat
from scipy.signal.waveforms import chirp

from acoustics_hardware.processors import deconvolve
from acoustics_hardware.serial import VariSphere

## Set up audio hardware

Adjust your settings for the individual audio hardware!

In [ ]:
# Show list of available audio devices
print(sd.query_devices())

# Audio device configuration
device_name = 'Orion 32'              # e.g. on macOS
# device_name = 'Orion32 ASIO Driver' # e.g. on Windows
fs = 48000 # in Hz
# output channel mapping (first channel in list will be used as loopback)
device_output_ch = [1, 2] # IDs starting from 1
# input channel mapping (first channel in list will be used as loopback)
device_input_ch  = [1, 2] # IDs starting from 1

# Sweep configuration
sweep_hp, sweep_lp = 50, 22e3    # in Hz
sweep_amplitude = -38            # in dB_FS
sweep_duration = [0.2, 2.0, 0.8] # in s, duration of [pre-silence, sweep, post-silence]

# Deconvolution configuration
deconv_ir_hp = 20     # in Hz
deconv_ir_len = 2**14 # in samples

print(f'\nCheck audio device "{device_name}" ... ', end='')
try:
    sd.check_output_settings(
        device=device_name,
        channels=len(device_output_ch), # as number
        samplerate=fs,                  # in Hz
    )
    sd.check_input_settings(
        device=device_name,
        channels=len(device_input_ch), # as number
        samplerate=fs,                 # in Hz
    )
except (ValueError, sd.PortAudioError) as e:
    sys.exit(e)
print('Done.')

print(f'\nGenerate sweep ... ', end='')
data_out = chirp(
    t=np.arange(np.ceil(sweep_duration[1] * fs)) / fs, # in s
    f0=sweep_hp,          # in Hz
    t1=sweep_duration[1], # in s
    f1=sweep_lp,          # in Hz
    method='logarithmic',
    phi=90,               # in deg
)                         # as [samples,]
# Apply amplitude scaling
data_out *= 10**(sweep_amplitude / 20) # as factor
# Prepend and append zeroes
data_out = np.pad(
    array=data_out,
    pad_width=(int(sweep_duration[0] * fs), int(sweep_duration[2] * fs)),
)
data_out_is_adjusted = False
print('Done.')

## Prepare functions for plotting

In [ ]:
# Prepare output widgets
widget_preview_str = '<h3><center><font color="green">{}</font></center></h3>'
widget_preview_ir = Output()
widget_preview_ir.layout.border = '3px solid green'
widget_preview_rec = Output()
widget_preview_rec.layout.border = '1px solid green'

def generate_time_estimation_string(duration, is_prediction=True):
    end_time = datetime.now()
    if is_prediction: end_time += duration
    return f'{duration} (completed at {end_time.strftime("%H:%M")})'
    
def generate_levels_string(data):
    _PEAK_THRESHOLD = -3.0 # in dB
    
    data_peak = 20 * np.log10(np.max(np.abs(data)))
    data_rms = np.sqrt(np.mean(np.power(data, 2), axis=-1)) # linear
    rms_str = np.array2string(
        20 * np.log10(np.abs(data_rms)),                    # in dB
        precision=1,
        sign='+',
        floatmode='fixed',
        separator=', ',
    )
    
    if data_peak > _PEAK_THRESHOLD:
        peak_str = np.array2string(
            20 * np.log10(np.max(np.abs(data), axis=-1)), # in dB
            precision=1,
            sign='+',
            floatmode='fixed',
            separator=', ',
        )
        print(
            f'Detected peak above {_PEAK_THRESHOLD:+.1f} dB_FS in {peak_str} dB',
            file=sys.stderr
        )
    
    return f'PEAK {data_peak:+.1f} dB, RMS {rms_str} dB'

def plot_grid(coords_az_el_r_rad, title=None, is_show_lines=False):
    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z
    
    # as [azimuth, elevation, radius] in rad
    coords_az_el_r_rad = np.atleast_2d(coords_az_el_r_rad) 
    # as (x, y, z) in ms
    x, y, z = sph2cart(coords_az_el_r_rad[0], coords_az_el_r_rad[1], coords_az_el_r_rad[2])
    
    plt.figure(title, figsize=(8, 8))
    ax = plt.gca(projection='3d')
    
    if is_show_lines:
        ax.plot3D(x, y, z)
        for pos in range(len(x)):
            ax.text(x[pos], y[pos], z[pos], s=pos, fontsize=14,
                    horizontalalignment='center', verticalalignment='center')
    else:
        ax.scatter3D(x, y, z, s=30, depthshade=True)
    
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('Z [m]')
    ax.set_box_aspect([1, 1, 1])
    ax.view_init(azim=-160, elev=25) # in deg
    
    plt.tight_layout()
    plt.show()

# Prevent 'divide by zero encountered in log10' warning
np.seterr(divide='ignore')

# noinspection PyShadowingNames
def plot_data(data, fs, ch_labels=None, is_stacked=False):
    _MAG_DR = [120, 80] # in dB if `is_stacked` or not
    
    data = np.atleast_2d(data)
    if ch_labels is not None:
        ch_labels = np.atleast_1d(ch_labels)
        if len(ch_labels) != data.shape[-2]:
            sys.exit(f'Mismatch in number of audio channels ({data.shape[-2]}) '
                     f'and number of provided labels {len(ch_labels)}.')
    if data.ndim == 3:
        # combine first and second dimension
        ch_labels = np.repeat(ch_labels, data.shape[0]) # also works for string lists
        data = np.reshape(data, (data.shape[0] * data.shape[1], -1), order='F')
        # as [[[channel0_iter0, samples], [channel0_iter1, samples], ...]
        
    n_ch, n_sample = data.shape
    t_data = np.linspace(0, n_sample / fs, n_sample) # in samples
    f_data = rfftfreq(n_sample, 1 / fs)              # in Hz
    spec_data = 20 * np.log10(np.abs(rfft(data)))    # in dB
    
    if is_stacked: n_ch = 1
        
    fig, axes = plt.subplots(
        nrows=n_ch,
        ncols=2 if is_stacked else 3,
        squeeze=False,
        sharex='col',
        figsize=(15, 5 if is_stacked else(3 * n_ch))
    )
    for ch in np.arange(n_ch): # individual channels
        ch_plot = (range(data.shape[0]) if is_stacked
                   else ch)
        
        # Time domain
        ax = axes[ch, 0]
        ax.plot(t_data, data[ch_plot, :].T)
        ax.set_xlim(0, n_sample / fs) # in s
        ax.grid()
        if ch_labels is not None:
            label = ([f'in_{l}' for l in ch_labels] if is_stacked
                     else [f'in_{ch_labels[ch]}'])
            ax.legend(label, loc='upper right')
        ax.set_xlabel('Time (s)' if ch >= n_ch - 1 else '')
        ax.set_ylabel('Amplitude')

        # Frequency domain
        y_max = np.ceil(spec_data[ch_plot, :].max() / 5) * 5
        ax = axes[ch, 1]
        ax.semilogx(f_data, spec_data[ch_plot, :].T)
        ax.set_xlim(20, fs/2)                                                  # in Hz
        ax.set_ylim(y_max - (_MAG_DR[0] if is_stacked else _MAG_DR[1]), y_max) # in dB
        ax.grid()
        ax.set_xlabel('Frequency (Hz)' if ch >= n_ch - 1 else '')
        ax.set_ylabel('Magnitude (dB)')
        
        if not is_stacked:
            # Spectrogram
            ax = axes[ch, 2]
            [specs, _, _, img] = ax.specgram(
                x=data[ch_plot, :].T,
                Fs=fs,        # in Hz
                NFFT=1024,    # in samples
                noverlap=512, # in samples
                mode='magnitude',
                scale='dB',
            )
            specs_peak = np.ceil(20 * np.log10(np.max(np.abs(specs))) / 5) * 5 # in dB
            ax.set_yscale('log')
            ax.set_ylim(100, fs/2) # in Hz
            ax.set_xlabel('Time (s)' if ch >= n_ch - 1 else '')
            ax.set_ylabel('Frequency (Hz)')
            plt.colorbar(mappable=img, ax=ax, label='Magnitude (dB)')
            img.set_clim(specs_peak - _MAG_DR[1], specs_peak) # in dB
    
    plt.show()

## Test audio

Execute this cell to test if the audio setup works.

In [ ]:
%matplotlib inline

# Determine latency if it has not been done yet
if not data_out_is_adjusted:
    print(f'\nRecord sweep ... ', end='')
    data_rec = sd.playrec(
        data=data_out,                   # as [samples,]
        samplerate=fs,                   # in Hz
        device=device_name,
        input_mapping=device_input_ch,   # channel numbers start from 1
        output_mapping=device_output_ch, # channel numbers start from 1
        blocking=True,
    ).T                                  # as [channels, samples]
    print('Done.')

    # Determine latency by initial zeroes in the recording
    # (this value will be used in the measurement section later)
    data_rec_latency = int(np.min(np.argmax(np.abs(data_rec) > 0, axis=-1))) # in samples
    display(Markdown(widget_preview_str.format(f'Adjust for detected latency of {data_rec_latency} samples')))

    # Append zeroes to excitation signal based on determined latency
    data_out = np.pad(data_out, pad_width=(0, data_rec_latency))
    data_out_is_adjusted = True
    sleep(1) # in s
else:
    # noinspection PyUnresolvedReferences
    display(Markdown(widget_preview_str.format(f'Using detected latency of {data_rec_latency} samples')))

print(f'\nRecord sweep ... ', end='')
data_rec = sd.playrec(
    data=data_out,                   # as [samples,]
    samplerate=fs,                   # in Hz
    device=device_name,
    input_mapping=device_input_ch,   # channel numbers start from 1
    output_mapping=device_output_ch, # channel numbers start from 1
    blocking=True,
).T                                  # as [channels, samples]
print('Done.')

# Cut initial zeroes based on determined latency
data_rec = data_rec[:, data_rec_latency:] # in samples

# Print recorded peak and individual RMS levels
print(generate_levels_string(data_rec))

display(Markdown('<h2><center>Excitation signal</center></h2>'))
plot_data(data_out, fs, ch_labels=device_input_ch[0])

display(Markdown('<h2><center>Recorded signals</center></h2>'))
for ch in range(data_rec.shape[0]):
    display(
        Markdown(f'<h3>Channel in_{device_input_ch[ch]}</h3>'),
        Audio(data_rec[ch, :], rate=fs),
    )
plot_data(data_rec, fs, ch_labels=device_input_ch)

data_ir = deconvolve(
    input=data_rec[0, :], # first channel as loopback
    output=data_rec[1:, :],
    fs=fs,                # in Hz
    f_low=20,             # in Hz
    f_high=sweep_lp,      # in Hz
    T=deconv_ir_len / fs, # in s
)                         # as [channels, samples]
if not data_ir.shape[0]:
    sys.exit('No deconvolved data available (only loopback channel recorded).')
    
display(Markdown('<h2><center>Deconvolved Impulse Responses</center></h2>'))
plot_data(data_ir, fs, ch_labels=device_input_ch[1:])
if data_ir.ndim > 1 and data_ir.shape[0] > 1:
    plot_data(data_ir, fs, ch_labels=device_input_ch[1:], is_stacked=True)

## Test VariSphere

Initialize the VariSphere and move the turntable to an arbitrary position.

In [ ]:
# VariSphere configuration
vari_ip = '192.168.127.120' # default value
# For a 1:1 Ethernet connection, set your local IPv4 to an adjacent address

print(f'\nConnect to VariSphere at {vari_ip} ... ', end='')
vari = VariSphere(
    az_port='4001', # default value
    el_port='4002', # default value
    ip=vari_ip,
)
try:
    vari.az.check_pc_mc_communication()
    vari.el.check_pc_mc_communication()
except Exception as e:
    sys.exit(e)
print('Done.')

# Try different angles (in deg) here
az, el = 0, 0 # in deg
print(f'\nMove VariSphere to {az=}°, {el=}° ... ', end='')
vari.move_blocking(az=az, el=el)
print('Done.')

## Prepare the SMA grid

Adjust your settings to the desired grid!

In [ ]:
%matplotlib widget

# Import requirement
from pyfar.spatial import samplings # from `pip install git+https://github.com/pyfar/pyfar.git`

# SMA grid configuration
array_r = 0.085                                  # in m
array_grid_type, array_sh_order = 'SMA_FM', 3  # generate Fliege-Maier grid, 16 positions
# array_grid_type, array_sh_order = 'SMA_FM', 5  # generate Fliege-Maier grid, 36 positions
# array_grid_type, array_sh_order = 'SMA_FM', 8  # generate Fliege-Maier grid, 81 positions
# array_grid_type, array_sh_order = 'SMA_FM', 12 # generate Fliege-Maier grid, 169 positions
# array_grid_type, array_sh_order = 'SMA_FM', 29 # generate Fliege-Maier grid, 900 positions
# array_grid_type, array_sh_order = 'SMA_LE', 3    # generate Lebedev grid, 26 positions
# array_grid_type, array_sh_order = 'SMA_LE', 5  # generate Lebedev grid, 50 positions
# array_grid_type, array_sh_order = 'SMA_LE', 8  # generate Lebedev grid, 110 positions
# array_grid_type, array_sh_order = 'SMA_LE', 12 # generate Lebedev grid, 230 positions
# array_grid_type, array_sh_order = 'SMA_LE', 29 # generate Lebedev grid, 1202 positions
# array_grid_type, array_sh_order = 'EMA', 3     # generate equatorial grid, 7 positions
# array_grid_type, array_sh_order = 'EMA', 5     # generate equatorial grid, 11 positions
# array_grid_type, array_sh_order = 'EMA', 8     # generate equatorial grid, 17 positions
# array_grid_type, array_sh_order = 'EMA', 12    # generate equatorial grid, 25 positions
# array_grid_type, array_sh_order = 'EMA', 29    # generate equatorial grid, 59 positions

# Determine spherical sampling grid
if 'EMA' in array_grid_type.upper():
    # Generate spherical coordinates
    array_coords = np.zeros((3, 2 * array_sh_order + 1)) # as [azimuth, elevation, radius] in rad
    array_coords[0] = np.linspace(start=0, stop=2 * np.pi, num=array_coords.shape[-1], endpoint=False)
    array_coords[2, :] = array_r
    
    # Generate quadrature weights (sum normalized to 1)
    array_weights = np.ones((array_coords.shape[-1])) / array_coords.shape[-1]
    
else:
    if 'LE' in array_grid_type.upper():
        array_grid = samplings.sph_lebedev(sh_order=array_sh_order, radius=array_r)
    elif 'FM' in array_grid_type.upper():
        array_grid = samplings.sph_fliege(sh_order=array_sh_order, radius=array_r)
    else:
        sys.exit(f'Unknown spherical sampling grid type "{array_grid_type}".')

    # Gather spherical coordinates
    array_coords = array_grid.get_sph(convention='top_elev', unit='rad').T # as [azimuth, elevation, radius] in rad
    
    # Gather quadrature weights
    array_weights = array_grid.weights
    del array_grid

# # Determine coordinate sort indeces by ascending azimuth
# # (this yields an inefficient path with respect to elevation movement)
# array_coords_sort_ids = np.argsort(array_coords[0])

# Determine coordinate sort indeces by ascending azimuth and elevation
# (this yields an okay path with respect to elevation movement)
array_coords_sort_ids = np.lexsort((array_coords[1], array_coords[0]))

# Plot grid raw and sorted
array_name = f'{array_grid_type.upper()}{array_coords.shape[-1]}'
plot_grid(coords_az_el_r_rad=array_coords, title=f'{array_name} raw')
plot_grid(
    coords_az_el_r_rad=array_coords[:, array_coords_sort_ids],
    title=f'{array_name} sorted',
    is_show_lines=True,
)

# Transform coordinates into deg
array_coords = np.rad2deg(array_coords)

## Perform the measurement series

Adjust your settings to the individual measurement conditions!

In [ ]:
%matplotlib inline

# Measurement configuration
out_file_name = f'data/{array_name}/{array_name}' + '_pos_{:04d}_az_{:03.0f}_el_{:03.0f}.mat'
meas_iterations = 2        # measurement iterations per position that will be averaged
vari_duration = [1.1, 1.0] # in s, _ESTIMATED_ duration of [move, post-halt-time]
post_duration = [2.0, 1.0] # in s, _ESTIMATED_ duration of post-processing [per iteration, per position]

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

# Check that the recording latency has been determined
try:
    data_rec_latency
except NameError:
    sys.exit('Recording latency is undefined. Execute the "Test Audio" section to determine it!')

# Print time estimation
total_iter = array_coords.shape[-1] * meas_iterations
total_meas = total_iter * (np.sum(vari_duration) + (data_out.shape[0] / fs) 
                           + post_duration[0] + (post_duration[1] / meas_iterations))
display(Markdown('<h2><center>Estimated measurement duration is {}</center></h2>'
                 .format(generate_time_estimation_string(timedelta(seconds=np.ceil(total_meas))))))
time_start = datetime.now()

display(widget_preview_ir)
widget_preview_ir.clear_output()
display(widget_preview_rec)
widget_preview_rec.clear_output()

# Wait before starting the measurement
sleep(1) # in s

try:
    for cur_id in range(len(array_coords_sort_ids)):
        grid_id = array_coords_sort_ids[cur_id]
        az, el = array_coords[0, grid_id], array_coords[1, grid_id]
        print(f'{cur_id=}, {grid_id=}, {az=}, {el=}')
        
        print(f'\nMove VariSphere to {az=:.1f}°, {el=:.1f}° ... ', end='')
        vari.move_blocking(az=az, el=el)
        print('Done.')

        # Wait for vibrations to settle
        sleep(vari_duration[1]) # in s

        # Prepare data arrays
        data_rec = np.zeros((meas_iterations, len(device_input_ch), data_out.shape[-1] - data_rec_latency))
        data_ir = np.zeros((meas_iterations, len(device_input_ch) - 1, deconv_ir_len))

        for i in range(meas_iterations):
            pos_str = f'{az=:.1f}°, {el=:.1f}° [{meas_iterations * cur_id + i + 1}/{total_iter}]'

            print(f'Record sweep for {pos_str} ... ', end='')
            rec = sd.playrec(            # as [iterations, channels, samples]
                data=data_out,                   # as [samples,]
                samplerate=fs,                   # in Hz
                device=device_name,
                input_mapping=device_input_ch,   # channel numbers start from 1
                output_mapping=device_output_ch, # channel numbers start from 1
                blocking=True,
            ).T                                  # as [channels, samples]
            print('Done.')

            # Cut initial zeroes based on determined latency
            data_rec[i, :] =  rec[:, data_rec_latency:] # in samples
            
            # Print recorded peak and individual RMS levels
            lvl_str = generate_levels_string(data_rec[i])
            print(lvl_str)

            data_ir[i] = deconvolve(  # as [iterations, channels-1, samples]
                input=data_rec[i, 0], # first channel as loopback
                output=data_rec[i, 1:],
                fs=fs,                # in Hz
                f_low=deconv_ir_hp,   # in Hz
                f_high=sweep_lp,      # in Hz
                T=deconv_ir_len / fs, # in s
            )                         # as [channels, samples]

            # Plot data preview (updated after each measurement)
            with widget_preview_rec:
                widget_preview_rec.clear_output(wait=True)
                plot_data(data_rec[i], fs, ch_labels=device_input_ch)
                display(Markdown(f'<h4><center>{lvl_str}</center></h4>'))
                display(Markdown(widget_preview_str.format(f'Preview individual signals for {pos_str}')))
#                     plot_data(data_ir[i], fs, ch_labels=device_input_ch[1:])
#                     display(Markdown(widget_preview_str.format(f'Preview individual IRs for {pos_str}')))
        
        # Average deconvolved IRs
        data_ir = np.mean(data_ir, axis=0) # as [channels-1, samples]

        # Plot averaged IRs preview (updated after each number of iterations)
        with widget_preview_ir:
            plot_is_stacked = data_ir.ndim > 1 and data_ir.shape[0] > 1
            widget_preview_ir.clear_output(wait=True)
            plot_data(data_ir, fs, ch_labels=device_input_ch[1:], is_stacked=plot_is_stacked)
            display(Markdown(widget_preview_str.format(f'Preview averaged IRs for {pos_str}')))

        # Store data
        file_name = out_file_name.format(grid_id, az, el)
        print(f'Save data to {file_name} ... ', end='')
        savemat(
            file_name,
            {
                'signal_structure': '[samples, channels, iterations]',
                'signal': data_rec.T,          # as [samples, channels, iterations]
                'ir': data_ir.T,               # as [samples, channels-1]
                'fs': fs,                      # in Hz
                'ch_output': device_output_ch, # as list
                'ch_input': device_input_ch,   # as list
                'sh_order': array_sh_order,    # as spherical harmonics order
                'grid_weight': array_weights[grid_id], # as quadrature weight
                'grid_id': grid_id,            # as original position index
                'meas_id': cur_id,             # as measurement position index
                'az_deg': az,                  # in deg
                'el_deg': el,                  # in deg
                'r': array_r,                  # in m
                'scatter_r': array_r,          # in m
            },
        )
        print('Done.')
        
    time_end = timedelta(seconds=np.ceil((datetime.now() - time_start).seconds))
    display(Markdown('<h2><center>Measurement completed after {}</center></h2>'
                     .format(generate_time_estimation_string(time_end, is_prediction=False))))
    
    print(f'\nReset VariSphere ... ', end='')
    vari.move_blocking(az=0, el=0)
    print('Done.')
    
except KeyboardInterrupt:
    print('Interrupted by user.', file=sys.stderr)